In [3]:
# Import libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import optuna

pd.set_option('display.max_columns', 500)
df = pd.read_csv("../data/processed/all_shots-v4.csv", index_col=0)
df = df[[ 'Shot Zone Basic_Above the Break 3', 'Shot Zone Basic_Backcourt', 'Shot Zone Basic_In The Paint (Non-RA)',
       'Shot Zone Basic_Left Corner 3', 'Shot Zone Basic_Mid-Range','Shot Zone Basic_Restricted Area', 'Shot Zone Basic_Right Corner 3',
       'Shot Zone Area_Back Court(BC)', 'Shot Zone Area_Center(C)', 'Shot Zone Area_Left Side Center(LC)', 'Shot Zone Area_Left Side(L)',
       'Shot Zone Area_Right Side Center(RC)', 'Shot Zone Area_Right Side(R)', 'Shot Zone Range_16-24 ft.', 'Shot Zone Range_24+ ft.',
       'Shot Zone Range_8-16 ft.', 'Shot Zone Range_Back Court Shot', 'Shot Zone Range_Less Than 8 ft.', 'target',
       'PERIOD', 'PLAYER1_NAME', 'at_home', 'PREVIOUS_OFF_REBOUND', 'PREVIOUS_DEF_REBOUND', 'dunk_shot',
       'PREVIOUS_OFF_TURNOVER', 'PREVIOUS_OFF_MISSED', 'DETAILLED_SHOT_TYPE',  'seconds_left', 'Age', 'TS%', 'PTM', 'ASTM', 'ORBM', 'STLM', 'BLKM', 'TOVM', 'USG%',
       'FG%', '2P%', '3P%', 'FT%', 'PTS', 'year_start', 'year_end', 'height', 'weight', 'C', 'PF', 'PG', 'PG-SG', 'SF', 'SF-SG', 'SG', 'SG-PG',
       'E_DEF_RATING', 'E_OFF_RATING']]

def update_shot_type(data):    
    data.loc[(data.DETAILLED_SHOT_TYPE != 'JUMP SHOT') & (data.DETAILLED_SHOT_TYPE != "FREE THROW"), 'DETAILLED_SHOT_TYPE'] = 'OTHER'
    return data

df = update_shot_type(df)

In [31]:
X_train = ""
y_train = ""
X_test = ""
y_test = ""


def for_player(player:None, year=None):
    global X_test
    global X_train
    global y_test
    global y_train

    if(player):
        df_player = df[df.PLAYER1_NAME==player]
        # ajouter la sélection par année
    else:
        df_player = df.copy()

    X = df_player.drop(['target', 'PLAYER1_NAME'], axis=1)
    X = pd.get_dummies(X)
    y = df_player.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, stratify=y)
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    skb = SelectKBest(score_func=chi2, k=15)
    skb.fit(X_train,y_train)
    X_train = skb.transform(X_train)
    X_test = skb.transform(X_test)

    scores = pd.DataFrame(skb.scores_)
    cols = pd.DataFrame(X.columns)
    featureScores = pd.concat([cols,scores],axis=1)
    featureScores.columns = ['feature','score']
    print(featureScores.nlargest(15, 'score'))

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # enregistrer le modèle sous le nom "nomJoueur_saison"
    # voir si on peut les appeler depuis stremlit


def objective(trial):
    global X_test
    global X_train
    global y_test
    global y_train
    

    n_estimators = trial.suggest_int('num_estimators', 500, 1000) 
    max_depth = trial.suggest_int('max_depth', 5, 15) 
    min_samples_split = trial.suggest_int('min_samples_split', 10, 30) 

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split)


    return np.mean(cross_val_score(model, X_train, y_train, cv=3, n_jobs=-1, scoring='accuracy'))


In [ ]:
for_player(None)

# Allen Iverson

In [33]:
for_player("Allen Iverson")

[I 2024-08-14 17:18:59,178] A new study created in memory with name: no-name-3ad77d23-e35c-455f-ab78-d5d1df9e4fec


                                  feature        score
55          DETAILLED_SHOT_TYPE_JUMP SHOT  1488.630543
54         DETAILLED_SHOT_TYPE_FREE THROW  1351.387573
15               Shot Zone Range_8-16 ft.   410.167581
8                Shot Zone Area_Center(C)   409.527092
14                Shot Zone Range_24+ ft.   361.661827
24                    PREVIOUS_OFF_MISSED   361.000407
0       Shot Zone Basic_Above the Break 3   327.783426
9     Shot Zone Area_Left Side Center(LC)   236.157953
2   Shot Zone Basic_In The Paint (Non-RA)   218.486224
13              Shot Zone Range_16-24 ft.   216.380398
10            Shot Zone Area_Left Side(L)   207.150456
56              DETAILLED_SHOT_TYPE_OTHER   141.451102
4               Shot Zone Basic_Mid-Range   138.544468
11   Shot Zone Area_Right Side Center(RC)   121.074406
21                   PREVIOUS_DEF_REBOUND   119.457279


[I 2024-08-14 17:19:08,779] Trial 0 finished with value: 0.7013528114838126 and parameters: {'num_estimators': 598, 'max_depth': 14, 'min_samples_split': 21}. Best is trial 0 with value: 0.7013528114838126.
[I 2024-08-14 17:19:17,870] Trial 1 finished with value: 0.7012934783878116 and parameters: {'num_estimators': 812, 'max_depth': 14, 'min_samples_split': 24}. Best is trial 0 with value: 0.7013528114838126.
[I 2024-08-14 17:19:26,805] Trial 2 finished with value: 0.7015308002124461 and parameters: {'num_estimators': 902, 'max_depth': 6, 'min_samples_split': 26}. Best is trial 2 with value: 0.7015308002124461.
[I 2024-08-14 17:19:33,172] Trial 3 finished with value: 0.7015308002124461 and parameters: {'num_estimators': 540, 'max_depth': 6, 'min_samples_split': 24}. Best is trial 2 with value: 0.7015308002124461.
[I 2024-08-14 17:19:41,407] Trial 4 finished with value: 0.7014121340204441 and parameters: {'num_estimators': 766, 'max_depth': 10, 'min_samples_split': 13}. Best is trial 2

Number of finished trials:  20
Best trial:
  Value: 0.701649466404448
  Params: 
    num_estimators: 928
    max_depth: 5
    min_samples_split: 26


# Stephen Curry

In [26]:
for_player("Stephen Curry")

[I 2024-08-14 11:28:55,535] A new study created in memory with name: no-name-65b0f1e3-1416-4100-b977-6af55ead2445


                                  feature        score
54         DETAILLED_SHOT_TYPE_FREE THROW  1472.750416
55          DETAILLED_SHOT_TYPE_JUMP SHOT  1217.672129
15               Shot Zone Range_8-16 ft.   835.641694
4               Shot Zone Basic_Mid-Range   467.454671
24                    PREVIOUS_OFF_MISSED   407.522940
0       Shot Zone Basic_Above the Break 3   388.701147
14                Shot Zone Range_24+ ft.   375.661194
8                Shot Zone Area_Center(C)   334.582803
9     Shot Zone Area_Left Side Center(LC)   158.469557
11   Shot Zone Area_Right Side Center(RC)   145.974069
21                   PREVIOUS_DEF_REBOUND   100.780700
2   Shot Zone Basic_In The Paint (Non-RA)    98.911290
7           Shot Zone Area_Back Court(BC)    85.016836
16        Shot Zone Range_Back Court Shot    85.016836
1               Shot Zone Basic_Backcourt    82.758309


[I 2024-08-14 11:28:57,974] Trial 0 finished with value: 0.689806227809135 and parameters: {'num_estimators': 584, 'max_depth': 7, 'min_samples_split': 24}. Best is trial 0 with value: 0.689806227809135.
[I 2024-08-14 11:29:02,264] Trial 1 finished with value: 0.6919694405121031 and parameters: {'num_estimators': 941, 'max_depth': 12, 'min_samples_split': 10}. Best is trial 1 with value: 0.6919694405121031.
[I 2024-08-14 11:29:04,773] Trial 2 finished with value: 0.6902117428131902 and parameters: {'num_estimators': 656, 'max_depth': 6, 'min_samples_split': 11}. Best is trial 1 with value: 0.6919694405121031.
[I 2024-08-14 11:29:07,606] Trial 3 finished with value: 0.6902794520040249 and parameters: {'num_estimators': 676, 'max_depth': 8, 'min_samples_split': 18}. Best is trial 1 with value: 0.6919694405121031.
[I 2024-08-14 11:29:11,246] Trial 4 finished with value: 0.6896710287284887 and parameters: {'num_estimators': 878, 'max_depth': 7, 'min_samples_split': 14}. Best is trial 1 wit

Number of finished trials:  20
Best trial:
  Value: 0.6921722391330724
  Params: 
    num_estimators: 636
    max_depth: 14
    min_samples_split: 21


# LeBron James

In [28]:
for_player("LeBron James")

[I 2024-08-14 17:11:50,602] A new study created in memory with name: no-name-adf0468c-d957-4424-91c0-573dd3a19fe9


                                  feature        score
55          DETAILLED_SHOT_TYPE_JUMP SHOT  2375.167942
22                              dunk_shot  1089.865138
14                Shot Zone Range_24+ ft.  1044.562792
54         DETAILLED_SHOT_TYPE_FREE THROW   990.152419
0       Shot Zone Basic_Above the Break 3   951.276274
8                Shot Zone Area_Center(C)   779.224110
5         Shot Zone Basic_Restricted Area   759.645813
13              Shot Zone Range_16-24 ft.   598.491842
9     Shot Zone Area_Left Side Center(LC)   555.646033
11   Shot Zone Area_Right Side Center(RC)   440.327323
17        Shot Zone Range_Less Than 8 ft.   409.159180
12           Shot Zone Area_Right Side(R)   371.244416
10            Shot Zone Area_Left Side(L)   329.813006
15               Shot Zone Range_8-16 ft.   303.146995
2   Shot Zone Basic_In The Paint (Non-RA)   295.567536


[I 2024-08-14 17:12:03,113] Trial 0 finished with value: 0.6849717302345922 and parameters: {'num_estimators': 647, 'max_depth': 8, 'min_samples_split': 23}. Best is trial 0 with value: 0.6849717302345922.
[I 2024-08-14 17:12:16,497] Trial 1 finished with value: 0.6849404929247495 and parameters: {'num_estimators': 799, 'max_depth': 8, 'min_samples_split': 28}. Best is trial 0 with value: 0.6849717302345922.
[I 2024-08-14 17:12:28,917] Trial 2 finished with value: 0.6849404929247495 and parameters: {'num_estimators': 789, 'max_depth': 11, 'min_samples_split': 25}. Best is trial 0 with value: 0.6849717302345922.
[I 2024-08-14 17:12:38,747] Trial 3 finished with value: 0.6849404929247495 and parameters: {'num_estimators': 587, 'max_depth': 8, 'min_samples_split': 30}. Best is trial 0 with value: 0.6849717302345922.
[I 2024-08-14 17:12:50,684] Trial 4 finished with value: 0.685284103333021 and parameters: {'num_estimators': 973, 'max_depth': 5, 'min_samples_split': 22}. Best is trial 4 wi

Number of finished trials:  20
Best trial:
  Value: 0.6854090525723925
  Params: 
    num_estimators: 891
    max_depth: 6
    min_samples_split: 13


# Kobe Bryant

In [139]:
for_player("Kobe Bryant")

[I 2024-08-14 11:02:14,800] A new study created in memory with name: no-name-3b6f1b3a-b332-4d32-aa5b-54b35a077c7a
[I 2024-08-14 11:02:14,821] Trial 0 finished with value: 0.7029730872199521 and parameters: {'booster': 'gblinear', 'lambda': 0.04802436079573508, 'alpha': 0.003857902389419084, 'subsample': 0.9652397112742943, 'colsample_bytree': 0.4152394407266735}. Best is trial 0 with value: 0.7029730872199521.
[I 2024-08-14 11:02:14,873] Trial 1 finished with value: 0.5536142031844441 and parameters: {'booster': 'dart', 'lambda': 0.3089027339972324, 'alpha': 0.038016552715796355, 'subsample': 0.7392118974280959, 'colsample_bytree': 0.20859259997603383, 'max_depth': 7, 'min_child_weight': 2, 'eta': 0.00012139578544128238, 'gamma': 4.4055228456380935e-08, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.3104909353809927, 'skip_drop': 2.451060349036886e-06}. Best is trial 0 with value: 0.7029730872199521.
[I 2024-08-14 11:02:14,937] Trial 2 

                                 feature        score
55         DETAILLED_SHOT_TYPE_JUMP SHOT  2675.976430
54        DETAILLED_SHOT_TYPE_FREE THROW  2503.550747
15              Shot Zone Range_8-16 ft.   962.612154
8               Shot Zone Area_Center(C)   924.917172
14               Shot Zone Range_24+ ft.   831.978113
0      Shot Zone Basic_Above the Break 3   772.575470
24                   PREVIOUS_OFF_MISSED   713.257002
13             Shot Zone Range_16-24 ft.   570.353412
22                             dunk_shot   474.798439
11  Shot Zone Area_Right Side Center(RC)   430.166613


[I 2024-08-14 11:02:15,027] Trial 4 finished with value: 0.7029730872199521 and parameters: {'booster': 'gblinear', 'lambda': 1.378515519224754e-06, 'alpha': 2.478310481086142e-06, 'subsample': 0.4159178329594851, 'colsample_bytree': 0.42429407662780527}. Best is trial 0 with value: 0.7029730872199521.
[I 2024-08-14 11:02:15,098] Trial 5 finished with value: 0.5536142031844441 and parameters: {'booster': 'dart', 'lambda': 4.052288726838896e-05, 'alpha': 3.1378413357427174e-06, 'subsample': 0.4398420264918718, 'colsample_bytree': 0.5604315134265536, 'max_depth': 7, 'min_child_weight': 5, 'eta': 1.1511294771323277e-05, 'gamma': 2.3977119256539726e-08, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.1732252944693769e-07, 'skip_drop': 0.0010991036101078977}. Best is trial 0 with value: 0.7029730872199521.
[I 2024-08-14 11:02:15,159] Trial 6 finished with value: 0.5536142031844441 and parameters: {'booster': 'dart', 'lambda': 0.00103881968221

Number of finished trials:  50
Best trial:
  Value: 0.7029730872199521
  Params: 
    booster: gblinear
    lambda: 0.04802436079573508
    alpha: 0.003857902389419084
    subsample: 0.9652397112742943
    colsample_bytree: 0.4152394407266735


Classe prédite,0.0,1.0
Classe réelle,,
0,2126,1042
1,1066,2863


# Dirk Nowitzki

In [140]:
for_player("Dirk Nowitzki")

[I 2024-08-14 11:02:33,676] A new study created in memory with name: no-name-22afc055-7436-4342-b0ab-0ff1a63cda1e
[I 2024-08-14 11:02:33,729] Trial 0 finished with value: 0.5762269938650306 and parameters: {'booster': 'dart', 'lambda': 1.2526309843781749e-05, 'alpha': 0.6992661697284024, 'subsample': 0.20726925250818484, 'colsample_bytree': 0.3333961026085518, 'max_depth': 9, 'min_child_weight': 4, 'eta': 7.652104056014341e-06, 'gamma': 0.0066208732282050636, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 2.607418103993885e-06, 'skip_drop': 4.8140062808123214e-06}. Best is trial 0 with value: 0.5762269938650306.
[I 2024-08-14 11:02:33,798] Trial 1 finished with value: 0.5762269938650306 and parameters: {'booster': 'dart', 'lambda': 1.0830814372197628e-07, 'alpha': 0.0004002732215010535, 'subsample': 0.34589301454875043, 'colsample_bytree': 0.4151153418252072, 'max_depth': 9, 'min_child_weight': 6, 'eta': 3.580441556568106e-08, 'gamma': 1.

                                 feature        score
54        DETAILLED_SHOT_TYPE_FREE THROW  2599.127948
55         DETAILLED_SHOT_TYPE_JUMP SHOT  1359.155714
24                   PREVIOUS_OFF_MISSED   768.643008
15              Shot Zone Range_8-16 ft.   741.942917
8               Shot Zone Area_Center(C)   682.421717
14               Shot Zone Range_24+ ft.   634.419990
0      Shot Zone Basic_Above the Break 3   569.567600
13             Shot Zone Range_16-24 ft.   273.321763
11  Shot Zone Area_Right Side Center(RC)   251.162534
21                  PREVIOUS_DEF_REBOUND   247.819041


[I 2024-08-14 11:02:33,928] Trial 3 finished with value: 0.5762269938650306 and parameters: {'booster': 'dart', 'lambda': 0.004678598074579548, 'alpha': 3.275719074702458e-08, 'subsample': 0.529942635374355, 'colsample_bytree': 0.3710222649389695, 'max_depth': 3, 'min_child_weight': 7, 'eta': 0.03879917206810029, 'gamma': 4.116927400478001e-05, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.0007634367451038295, 'skip_drop': 3.2631383008192907e-06}. Best is trial 2 with value: 0.6644171779141105.
[I 2024-08-14 11:02:33,984] Trial 4 finished with value: 0.5762269938650306 and parameters: {'booster': 'gbtree', 'lambda': 9.448565766778477e-06, 'alpha': 1.2265784430813582e-06, 'subsample': 0.6589557505740078, 'colsample_bytree': 0.814533641898975, 'max_depth': 5, 'min_child_weight': 10, 'eta': 2.0981286860197612e-06, 'gamma': 6.746842673769997e-05, 'grow_policy': 'lossguide'}. Best is trial 2 with value: 0.6644171779141105.
[I 2024-08-14 11:

Number of finished trials:  50
Best trial:
  Value: 0.6662576687116565
  Params: 
    booster: gblinear
    lambda: 4.170219620733197e-06
    alpha: 0.0034761328740172356
    subsample: 0.27529517144501015
    colsample_bytree: 0.6476066894569764


Classe prédite,0.0,1.0
Classe réelle,,
0,1844,919
1,1269,2488


# Dwyane Wade 

In [72]:
for_player("Dwyane Wade")

[I 2024-08-14 10:17:25,616] A new study created in memory with name: no-name-80613d3b-6257-4a0d-be1f-4bf6f5c25a2a
[I 2024-08-14 10:17:25,646] Trial 0 finished with value: 0.6685827195976289 and parameters: {'booster': 'gblinear', 'lambda': 0.012357977946566892, 'alpha': 0.01050302457598988, 'subsample': 0.8973798781463489, 'colsample_bytree': 0.5064857954301459}. Best is trial 0 with value: 0.6685827195976289.
[I 2024-08-14 10:17:25,716] Trial 1 finished with value: 0.6693012394467397 and parameters: {'booster': 'dart', 'lambda': 0.002733486841724965, 'alpha': 3.809249858603545e-07, 'subsample': 0.9902102795638019, 'colsample_bytree': 0.24982060029436265, 'max_depth': 5, 'min_child_weight': 8, 'eta': 8.099349277741993e-07, 'gamma': 7.780943628103804e-08, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 9.859911645077958e-05, 'skip_drop': 0.0013859867787690502}. Best is trial 1 with value: 0.6693012394467397.
[I 2024-08-14 10:17:25,787] Trial

Number of finished trials:  50
Best trial:
  Value: 0.6730734686545716
  Params: 
    booster: dart
    lambda: 3.9867514217454393e-05
    alpha: 9.837059112769506e-08
    subsample: 0.4868558177228015
    colsample_bytree: 0.5058937020929823
    max_depth: 5
    min_child_weight: 4
    eta: 0.2527222864061563
    gamma: 2.112980165806881e-08
    grow_policy: depthwise
    sample_type: weighted
    normalize_type: forest
    rate_drop: 0.003987975840841047
    skip_drop: 2.4942762980191427e-08


Classe prédite,0.0,1.0
Classe réelle,,
0,1478,960
1,891,2238


# Paul Pierce

In [73]:
for_player("Paul Pierce")

[I 2024-08-14 10:17:33,198] A new study created in memory with name: no-name-89fad0e0-5858-439c-8d41-1d5168240f8c
[I 2024-08-14 10:17:33,259] Trial 0 finished with value: 0.6809402332361516 and parameters: {'booster': 'gbtree', 'lambda': 0.0015062357133781056, 'alpha': 3.010876830104278e-05, 'subsample': 0.9879988907748902, 'colsample_bytree': 0.20509737518903137, 'max_depth': 7, 'min_child_weight': 10, 'eta': 0.00011686612024460278, 'gamma': 0.06432841957930999, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6809402332361516.
[I 2024-08-14 10:17:33,325] Trial 1 finished with value: 0.6909620991253644 and parameters: {'booster': 'gbtree', 'lambda': 3.6870937571390296e-07, 'alpha': 5.771326035807176e-06, 'subsample': 0.863354265242088, 'colsample_bytree': 0.5916610437105414, 'max_depth': 5, 'min_child_weight': 3, 'eta': 0.00035577882595941085, 'gamma': 0.5541178425100881, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 0.6909620991253644.
[I 2024-08-14 10:17:33,412]

Number of finished trials:  50
Best trial:
  Value: 0.6924198250728864
  Params: 
    booster: gblinear
    lambda: 2.119819931620608e-06
    alpha: 0.000296981984166381
    subsample: 0.6388902669354078
    colsample_bytree: 0.5230999459522406


Classe prédite,0.0,1.0
Classe réelle,,
0,1591,843
1,850,2204


# Tim Duncan

In [74]:
for_player("Tim Duncan")

[I 2024-08-14 10:17:37,813] A new study created in memory with name: no-name-bfa6ca7e-b98d-471b-9570-cafbce770fce
[I 2024-08-14 10:17:37,902] Trial 0 finished with value: 0.6655013799448022 and parameters: {'booster': 'dart', 'lambda': 1.220350675453576e-07, 'alpha': 4.9754787609151494e-05, 'subsample': 0.43996764103217156, 'colsample_bytree': 0.8097393506379016, 'max_depth': 5, 'min_child_weight': 3, 'eta': 0.00022747596305949924, 'gamma': 7.07652027457247e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.006256553007727671, 'skip_drop': 0.3571624930765526}. Best is trial 0 with value: 0.6655013799448022.
[I 2024-08-14 10:17:37,942] Trial 1 finished with value: 0.6638454461821527 and parameters: {'booster': 'gblinear', 'lambda': 0.009839479412592337, 'alpha': 0.0007423397479554105, 'subsample': 0.6937749651403586, 'colsample_bytree': 0.6273201939512765}. Best is trial 0 with value: 0.6655013799448022.
[I 2024-08-14 10:17:38,033] Tria

Number of finished trials:  50
Best trial:
  Value: 0.6673413063477461
  Params: 
    booster: dart
    lambda: 0.26174352836259157
    alpha: 5.194416522481589e-08
    subsample: 0.7333992882044708
    colsample_bytree: 0.9970460884981013
    max_depth: 3
    min_child_weight: 6
    eta: 0.002487557602306387
    gamma: 0.0004369611821928495
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: forest
    rate_drop: 0.019325303051792257
    skip_drop: 0.03784114947726144


Classe prédite,0.0,1.0
Classe réelle,,
0,1264,1134
1,676,2361


# Russell Westbrook

In [75]:
for_player("Russell Westbrook")

[I 2024-08-14 10:17:47,348] A new study created in memory with name: no-name-91546a2b-b3b8-415b-88c3-bcdab211cfae
[I 2024-08-14 10:17:47,378] Trial 0 finished with value: 0.6530651711467801 and parameters: {'booster': 'gblinear', 'lambda': 3.9672708334252926e-07, 'alpha': 5.5630283510267997e-08, 'subsample': 0.22710303426056067, 'colsample_bytree': 0.4801215358596486}. Best is trial 0 with value: 0.6530651711467801.
[I 2024-08-14 10:17:47,484] Trial 1 finished with value: 0.6604138464513634 and parameters: {'booster': 'gbtree', 'lambda': 0.005030202528261699, 'alpha': 0.5712602577129596, 'subsample': 0.8574659834121652, 'colsample_bytree': 0.8680582509514567, 'max_depth': 7, 'min_child_weight': 7, 'eta': 0.0001462421476001719, 'gamma': 2.429820343047201e-06, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 0.6604138464513634.
[I 2024-08-14 10:17:47,589] Trial 2 finished with value: 0.6567395087990717 and parameters: {'booster': 'gbtree', 'lambda': 1.3434154547058829e-08, 'alpha

Number of finished trials:  50
Best trial:
  Value: 0.664088184103655
  Params: 
    booster: gbtree
    lambda: 0.1553067226609043
    alpha: 9.40374424621823e-06
    subsample: 0.42296504304333504
    colsample_bytree: 0.9196525912745434
    max_depth: 7
    min_child_weight: 8
    eta: 0.0002509194582208215
    gamma: 0.00011316904210459366
    grow_policy: lossguide


Classe prédite,0.0,1.0
Classe réelle,,
0,1548,868
1,871,1884


# James Harden

In [76]:
for_player("James Harden")

[I 2024-08-14 10:17:57,151] A new study created in memory with name: no-name-065220bf-adc8-48bc-a01b-e52c36a443f9
[I 2024-08-14 10:17:57,244] Trial 0 finished with value: 0.6903124385930438 and parameters: {'booster': 'gbtree', 'lambda': 0.0006011248661312587, 'alpha': 0.0024255164135970893, 'subsample': 0.5880291880417152, 'colsample_bytree': 0.8528576002845532, 'max_depth': 5, 'min_child_weight': 4, 'eta': 0.0008053346379071104, 'gamma': 5.2594229382369815e-08, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.6903124385930438.
[I 2024-08-14 10:17:57,283] Trial 1 finished with value: 0.6677146787188053 and parameters: {'booster': 'gblinear', 'lambda': 4.9546083793820295e-08, 'alpha': 4.0527377390279514e-07, 'subsample': 0.5647170597133129, 'colsample_bytree': 0.6721186271014662}. Best is trial 0 with value: 0.6903124385930438.
[I 2024-08-14 10:17:57,369] Trial 2 finished with value: 0.6659461583808214 and parameters: {'booster': 'gbtree', 'lambda': 0.45583386549064653, 'alph

Number of finished trials:  50
Best trial:
  Value: 0.6952249950874435
  Params: 
    booster: gbtree
    lambda: 1.0852501924593756e-06
    alpha: 1.7338599320685001e-07
    subsample: 0.5345273745486468
    colsample_bytree: 0.9204779094744939
    max_depth: 9
    min_child_weight: 8
    eta: 5.18562267859712e-06
    gamma: 0.023924013061419093
    grow_policy: lossguide


Classe prédite,0.0,1.0
Classe réelle,,
0,1747,377
1,1252,1713


# Kevin Durant  

In [77]:
for_player("Kevin Durant")

[I 2024-08-14 10:18:04,123] A new study created in memory with name: no-name-99968152-63aa-4d37-b7f1-1a9a9e51753a
[I 2024-08-14 10:18:04,164] Trial 0 finished with value: 0.6976006345429308 and parameters: {'booster': 'gblinear', 'lambda': 2.3315396289261827e-07, 'alpha': 0.004068696957297376, 'subsample': 0.45697637988288076, 'colsample_bytree': 0.22190899379445173}. Best is trial 0 with value: 0.6976006345429308.
[I 2024-08-14 10:18:04,207] Trial 1 finished with value: 0.701764822526274 and parameters: {'booster': 'gblinear', 'lambda': 0.006829057343696372, 'alpha': 0.09166689393654334, 'subsample': 0.8939893185908987, 'colsample_bytree': 0.20509997536030983}. Best is trial 1 with value: 0.701764822526274.
[I 2024-08-14 10:18:04,331] Trial 2 finished with value: 0.6930398572278406 and parameters: {'booster': 'dart', 'lambda': 0.0004497226272072256, 'alpha': 1.540547741846339e-05, 'subsample': 0.6822583835154129, 'colsample_bytree': 0.4706504431093128, 'max_depth': 7, 'min_child_weigh

Number of finished trials:  50
Best trial:
  Value: 0.7043426531826293
  Params: 
    booster: gblinear
    lambda: 0.00012691551501936953
    alpha: 0.023537359194423693
    subsample: 0.8415875720430429
    colsample_bytree: 0.7289899256415275


Classe prédite,0.0,1.0
Classe réelle,,
0,1350,627
1,916,2150


# Kevin Garnett

In [103]:
for_player("Kevin Garnett")

[I 2024-08-14 10:41:33,705] A new study created in memory with name: no-name-a73e6e7d-da06-4d56-8b60-8881f5203ef8
[I 2024-08-14 10:41:33,821] Trial 0 finished with value: 0.5682789651293588 and parameters: {'booster': 'gbtree', 'lambda': 1.2616026830457587e-07, 'alpha': 1.3511122933406292e-08, 'subsample': 0.977221185319813, 'colsample_bytree': 0.35739490952781594, 'max_depth': 9, 'min_child_weight': 5, 'eta': 0.010166090277326515, 'gamma': 0.02721775325734419, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.5682789651293588.
[I 2024-08-14 10:41:33,878] Trial 1 finished with value: 0.666591676040495 and parameters: {'booster': 'gblinear', 'lambda': 0.08611333309903643, 'alpha': 0.009624060607561843, 'subsample': 0.21673263029211887, 'colsample_bytree': 0.8196867450469578}. Best is trial 1 with value: 0.666591676040495.
[I 2024-08-14 10:41:33,975] Trial 2 finished with value: 0.5682789651293588 and parameters: {'booster': 'gbtree', 'lambda': 9.197810149646376e-08, 'alpha': 1.

Number of finished trials:  50
Best trial:
  Value: 0.6681664791901012
  Params: 
    booster: gblinear
    lambda: 2.7451392518300194e-06
    alpha: 4.570006811017244e-08
    subsample: 0.3061566420099804
    colsample_bytree: 0.885455488880624


Classe prédite,1.0
Classe réelle,
0,1919
1,2526


# Dwight Howard 

In [102]:
for_player("Dwight Howard")

[I 2024-08-14 10:41:21,772] A new study created in memory with name: no-name-1de55108-3d32-4c2d-bc77-9b302401ffbd
[I 2024-08-14 10:41:21,823] Trial 0 finished with value: 0.6316410861865407 and parameters: {'booster': 'gblinear', 'lambda': 0.08442110855303028, 'alpha': 0.0001013751714722715, 'subsample': 0.7051484020024146, 'colsample_bytree': 0.7985632926720401}. Best is trial 0 with value: 0.6316410861865407.
[I 2024-08-14 10:41:21,873] Trial 1 finished with value: 0.6278630460448642 and parameters: {'booster': 'gblinear', 'lambda': 0.00015603361133009698, 'alpha': 2.6820197151982034e-08, 'subsample': 0.5900913077727319, 'colsample_bytree': 0.7955139312056418}. Best is trial 0 with value: 0.6316410861865407.
[I 2024-08-14 10:41:21,983] Trial 2 finished with value: 0.6070838252656434 and parameters: {'booster': 'gbtree', 'lambda': 1.607013045661884e-07, 'alpha': 0.14489786122991027, 'subsample': 0.9377648017458984, 'colsample_bytree': 0.802153623469894, 'max_depth': 7, 'min_child_weig

Number of finished trials:  50
Best trial:
  Value: 0.6337662337662338
  Params: 
    booster: gblinear
    lambda: 0.009448437953145315
    alpha: 0.008070247491543812
    subsample: 0.7419139100682692
    colsample_bytree: 0.2207231934183903


Classe prédite,1.0
Classe réelle,
0,1800
1,2435


# Shaquille O'Neal

In [101]:
for_player("Shaquille O'Neal")

[I 2024-08-14 10:41:06,738] A new study created in memory with name: no-name-d214c3ba-e4c6-486c-bfa0-ab41278999e6
[I 2024-08-14 10:41:06,890] Trial 0 finished with value: 0.5571045576407506 and parameters: {'booster': 'dart', 'lambda': 0.00011404478123766745, 'alpha': 2.7413644437347e-05, 'subsample': 0.9781061852430224, 'colsample_bytree': 0.7091708834213892, 'max_depth': 9, 'min_child_weight': 5, 'eta': 1.862584309315124e-05, 'gamma': 1.7179729186083822e-07, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.0003437159122138915, 'skip_drop': 0.17431997257429643}. Best is trial 0 with value: 0.5571045576407506.
[I 2024-08-14 10:41:06,940] Trial 1 finished with value: 0.6260053619302949 and parameters: {'booster': 'gblinear', 'lambda': 0.05973200339574233, 'alpha': 0.005328349932207069, 'subsample': 0.682479896022905, 'colsample_bytree': 0.7743744722025219}. Best is trial 1 with value: 0.6260053619302949.
[I 2024-08-14 10:41:07,022] Trial 2

Number of finished trials:  50
Best trial:
  Value: 0.6431635388739947
  Params: 
    booster: gblinear
    lambda: 3.435561204927551e-08
    alpha: 1.2968578781054434e-06
    subsample: 0.624065278197951
    colsample_bytree: 0.5550628627144668


Classe prédite,0.0,1.0
Classe réelle,,
0,837,815
1,537,1541


# Steve Nash   

In [100]:
for_player("Steve Nash")

[I 2024-08-14 10:40:58,713] A new study created in memory with name: no-name-0d9e973f-bf1d-427e-bc88-765ba3d79c47
[I 2024-08-14 10:40:58,787] Trial 0 finished with value: 0.5822825735531846 and parameters: {'booster': 'dart', 'lambda': 0.5031097546887068, 'alpha': 4.999178758925571e-05, 'subsample': 0.23225664808033458, 'colsample_bytree': 0.20899032052884145, 'max_depth': 5, 'min_child_weight': 3, 'eta': 2.982828326455909e-05, 'gamma': 0.21496566756976204, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.0011436399885287449, 'skip_drop': 0.0002107647925101928}. Best is trial 0 with value: 0.5822825735531846.
[I 2024-08-14 10:40:58,883] Trial 1 finished with value: 0.5822825735531846 and parameters: {'booster': 'dart', 'lambda': 0.0010155014273944016, 'alpha': 0.0033291806527751006, 'subsample': 0.3560245804926906, 'colsample_bytree': 0.621582039642623, 'max_depth': 5, 'min_child_weight': 4, 'eta': 5.951368343363313e-07, 'gamma': 0.250443

Number of finished trials:  50
Best trial:
  Value: 0.6763659877141933
  Params: 
    booster: gblinear
    lambda: 7.3715775100015e-08
    alpha: 0.0038763171102685606
    subsample: 0.27021846299488506
    colsample_bytree: 0.25848971865543635


Classe prédite,0.0,1.0
Classe réelle,,
0,920,372
1,644,1157


# Giannis Antetokounmpo

In [99]:
for_player("Giannis Antetokounmpo")

[I 2024-08-14 10:40:51,978] A new study created in memory with name: no-name-e2d7ee48-e9a2-4f17-9c29-2593e95d6f86
[I 2024-08-14 10:40:52,012] Trial 0 finished with value: 0.6966551326412919 and parameters: {'booster': 'gblinear', 'lambda': 3.778655220263377e-08, 'alpha': 9.150959592194782e-05, 'subsample': 0.6649913468193385, 'colsample_bytree': 0.7514005357849853}. Best is trial 0 with value: 0.6966551326412919.
[I 2024-08-14 10:40:52,080] Trial 1 finished with value: 0.6962706651287967 and parameters: {'booster': 'gbtree', 'lambda': 5.586059294888939e-08, 'alpha': 1.5510539655944222e-05, 'subsample': 0.6817089384890804, 'colsample_bytree': 0.6561390428794407, 'max_depth': 5, 'min_child_weight': 9, 'eta': 0.15341641637549078, 'gamma': 0.03909178253741281, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6966551326412919.
[I 2024-08-14 10:40:52,117] Trial 2 finished with value: 0.590157631680123 and parameters: {'booster': 'gblinear', 'lambda': 1.8746493016820378e-07, 'alpha'

Number of finished trials:  50
Best trial:
  Value: 0.6978085351787774
  Params: 
    booster: gblinear
    lambda: 1.0579354096079244e-08
    alpha: 1.0266296460549005e-07
    subsample: 0.7585538902062741
    colsample_bytree: 0.5982053047125827


Classe prédite,1.0
Classe réelle,
0,1066
1,1535


# Kawhi Leonard

In [98]:
for_player("Kawhi Leonard")

[I 2024-08-14 10:40:43,175] A new study created in memory with name: no-name-e8c923c5-034e-4738-a8c9-b97afa460da0
[I 2024-08-14 10:40:43,209] Trial 0 finished with value: 0.6815881666017906 and parameters: {'booster': 'gblinear', 'lambda': 0.007868698313307778, 'alpha': 4.379369446461671e-05, 'subsample': 0.46365986201523424, 'colsample_bytree': 0.379667901787355}. Best is trial 0 with value: 0.6815881666017906.
[I 2024-08-14 10:40:43,320] Trial 1 finished with value: 0.585441806150253 and parameters: {'booster': 'dart', 'lambda': 0.001691533497991901, 'alpha': 2.563273382717828e-06, 'subsample': 0.7089999882949023, 'colsample_bytree': 0.9689182335931215, 'max_depth': 7, 'min_child_weight': 10, 'eta': 0.00010095453331311146, 'gamma': 2.7044316489329328e-08, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.1005848904155136, 'skip_drop': 0.036559572096606}. Best is trial 0 with value: 0.6815881666017906.
[I 2024-08-14 10:40:43,410] Trial 2 f

Number of finished trials:  50
Best trial:
  Value: 0.6835344492020241
  Params: 
    booster: gblinear
    lambda: 0.02145251157997652
    alpha: 3.7887418512521986e-05
    subsample: 0.44452530665108325
    colsample_bytree: 0.39614077153157307


Classe prédite,1.0
Classe réelle,
0,1065
1,1504


# Jimmy Butler

In [97]:

for_player("Jimmy Butler")

[I 2024-08-14 10:40:32,986] A new study created in memory with name: no-name-bddab4df-ce3a-47cc-9117-e911eba9074b
[I 2024-08-14 10:40:33,054] Trial 0 finished with value: 0.5789892558694787 and parameters: {'booster': 'dart', 'lambda': 0.578575715042266, 'alpha': 3.2711328797951365e-08, 'subsample': 0.3691091235699026, 'colsample_bytree': 0.5238671034814836, 'max_depth': 5, 'min_child_weight': 8, 'eta': 6.046936785090119e-08, 'gamma': 0.08549603242286324, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 2.4519105215595653e-05, 'skip_drop': 0.27865886122793426}. Best is trial 0 with value: 0.5789892558694787.
[I 2024-08-14 10:40:33,111] Trial 1 finished with value: 0.6991643454038997 and parameters: {'booster': 'gbtree', 'lambda': 0.04902880703010301, 'alpha': 1.687535138126424e-08, 'subsample': 0.9935012587101992, 'colsample_bytree': 0.5403573793052536, 'max_depth': 5, 'min_child_weight': 6, 'eta': 0.10436793073838267, 'gamma': 0.1113223071

Number of finished trials:  50
Best trial:
  Value: 0.703541583764425
  Params: 
    booster: gblinear
    lambda: 0.11476614716354477
    alpha: 0.0005782861336198039
    subsample: 0.9428362449129021
    colsample_bytree: 0.753840607939094


Classe prédite,1.0
Classe réelle,
0,1058
1,1455


# Chris Webber

In [96]:
for_player("Chris Webber")

[I 2024-08-14 10:40:23,148] A new study created in memory with name: no-name-290d26f4-df15-404f-8a56-382b6c61eb68
[I 2024-08-14 10:40:23,174] Trial 0 finished with value: 0.6347714651858181 and parameters: {'booster': 'gblinear', 'lambda': 0.162960153814689, 'alpha': 1.9948412155901235e-05, 'subsample': 0.5106412026540326, 'colsample_bytree': 0.5395400910951476}. Best is trial 0 with value: 0.6347714651858181.
[I 2024-08-14 10:40:23,232] Trial 1 finished with value: 0.6416061512174285 and parameters: {'booster': 'gbtree', 'lambda': 6.059852043279718e-06, 'alpha': 4.298409025995614e-05, 'subsample': 0.3483909247463347, 'colsample_bytree': 0.6362288950248134, 'max_depth': 5, 'min_child_weight': 5, 'eta': 0.06465342887414718, 'gamma': 6.733776694769018e-07, 'grow_policy': 'lossguide'}. Best is trial 1 with value: 0.6416061512174285.
[I 2024-08-14 10:40:23,310] Trial 2 finished with value: 0.5079026057240495 and parameters: {'booster': 'dart', 'lambda': 0.3374819221274656, 'alpha': 0.00169

Number of finished trials:  50
Best trial:
  Value: 0.6510038445108928
  Params: 
    booster: gbtree
    lambda: 7.14931632490132e-08
    alpha: 0.0391379042359863
    subsample: 0.6026621540683557
    colsample_bytree: 0.5395645734273141
    max_depth: 5
    min_child_weight: 3
    eta: 0.04527881983457275
    gamma: 0.0027058548625063577
    grow_policy: lossguide


Classe prédite,1.0
Classe réelle,
0,1152
1,1189


# Jason Kidd 

In [95]:
for_player("Jason Kidd")

[I 2024-08-14 10:40:16,126] A new study created in memory with name: no-name-995c9bc5-a6b7-482e-8f4a-8b32c2b5f92d
[I 2024-08-14 10:40:16,143] Trial 0 finished with value: 0.6513409961685823 and parameters: {'booster': 'gblinear', 'lambda': 0.6916595301214273, 'alpha': 0.017594472858202186, 'subsample': 0.8123459425209822, 'colsample_bytree': 0.5418128332326149}. Best is trial 0 with value: 0.6513409961685823.
[I 2024-08-14 10:40:16,162] Trial 1 finished with value: 0.6513409961685823 and parameters: {'booster': 'gblinear', 'lambda': 0.2685127831986303, 'alpha': 1.0383215433187292e-08, 'subsample': 0.5287175731268314, 'colsample_bytree': 0.48019526167316046}. Best is trial 0 with value: 0.6513409961685823.
[I 2024-08-14 10:40:16,213] Trial 2 finished with value: 0.5325670498084292 and parameters: {'booster': 'dart', 'lambda': 0.25246833817745296, 'alpha': 0.0010571815209935204, 'subsample': 0.5252208520176322, 'colsample_bytree': 0.5092102265315652, 'max_depth': 9, 'min_child_weight': 8

Number of finished trials:  50
Best trial:
  Value: 0.6551724137931034
  Params: 
    booster: gblinear
    lambda: 0.001863215770115575
    alpha: 1.7228183221352768e-05
    subsample: 0.3569755775266368
    colsample_bytree: 0.5386858744792207


Classe prédite,0.0
Classe réelle,
0,556
1,488
